# Manual 3D Mesh Creation

This notebook demonstrates how to manually create a 3×3×3 cube mesh using Netgen's low-level meshing API.

In [1]:
from netgen.csg import unit_cube, Pnt
from netgen.meshing import Element1D, Element2D, Element3D, MeshPoint, FaceDescriptor, Mesh
from ngsolve.webgui import Draw
import ngsolve

In [2]:
# Parameters for 3x3x3 cube
N = 5

In [3]:
mesh = Mesh()
mesh.SetGeometry(unit_cube)
mesh.dim = 3

In [4]:
# Create all mesh points in 3D grid
pnums = []
for k in range(N + 1):  # z direction
    for j in range(N + 1):  # y direction
        for i in range(N + 1):  # x direction
            pnums.append(mesh.Add(MeshPoint(Pnt(i / N, j / N, k / N))))

print(f"Created {len(pnums)} points for a {N+1}×{N+1}×{N+1} grid")

Created 216 points for a 6×6×6 grid


In [5]:
# Helper function to get point index
def get_point_index(i, j, k):
    return i + j * (N + 1) + k * (N + 1) * (N + 1)

# Test the helper function
print(f"Point at (0,0,0): index {get_point_index(0,0,0)}")
print(f"Point at ({N},{N},{N}): index {get_point_index(N,N,N)}")

Point at (0,0,0): index 0
Point at (5,5,5): index 215


In [6]:
mesh.SetMaterial(1, "mat")

# Add FaceDescriptor first and get its number (for boundary faces)
fd_boundary = mesh.Add(FaceDescriptor(surfnr=1, domin=1, bc=1))
print(f"Created FaceDescriptor with index: {fd_boundary}")

Created FaceDescriptor with index: 1


In [7]:
# Create 3D elements (hexahedra/cubes)
element_count = 0
for k in range(N):
    for j in range(N):
        for i in range(N):
            # Define the 8 vertices of a hexahedron in correct order
            p1 = pnums[get_point_index(i, j, k)]
            p2 = pnums[get_point_index(i+1, j, k)]
            p3 = pnums[get_point_index(i+1, j+1, k)]
            p4 = pnums[get_point_index(i, j+1, k)]
            p5 = pnums[get_point_index(i, j, k+1)]
            p6 = pnums[get_point_index(i+1, j, k+1)]
            p7 = pnums[get_point_index(i+1, j+1, k+1)]
            p8 = pnums[get_point_index(i, j+1, k+1)]
            
            # Add hexahedral element
            mesh.Add(Element3D(1, [p1, p2, p3, p4, p5, p6, p7, p8]))
            element_count += 1

print(f"Created {element_count} hexahedral elements")

Created 125 hexahedral elements


In [8]:
# Add boundary faces (6 faces of the cube)
face_count = 0

# Bottom face (z=0)
for j in range(N):
    for i in range(N):
        p1 = pnums[get_point_index(i, j, 0)]
        p2 = pnums[get_point_index(i+1, j, 0)]
        p3 = pnums[get_point_index(i+1, j+1, 0)]
        p4 = pnums[get_point_index(i, j+1, 0)]
        mesh.Add(Element2D(fd_boundary, [p1, p2, p3, p4]))
        face_count += 1

# Top face (z=N)
for j in range(N):
    for i in range(N):
        p1 = pnums[get_point_index(i, j, N)]
        p2 = pnums[get_point_index(i+1, j, N)]
        p3 = pnums[get_point_index(i+1, j+1, N)]
        p4 = pnums[get_point_index(i, j+1, N)]
        mesh.Add(Element2D(fd_boundary, [p1, p4, p3, p2]))  # Reversed order for outward normal
        face_count += 1

print(f"Added bottom and top faces: {face_count} faces so far")

Added bottom and top faces: 50 faces so far


In [9]:
# Front face (y=0)
for k in range(N):
    for i in range(N):
        p1 = pnums[get_point_index(i, 0, k)]
        p2 = pnums[get_point_index(i+1, 0, k)]
        p3 = pnums[get_point_index(i+1, 0, k+1)]
        p4 = pnums[get_point_index(i, 0, k+1)]
        mesh.Add(Element2D(fd_boundary, [p1, p4, p3, p2]))
        face_count += 1

# Back face (y=N)
for k in range(N):
    for i in range(N):
        p1 = pnums[get_point_index(i, N, k)]
        p2 = pnums[get_point_index(i+1, N, k)]
        p3 = pnums[get_point_index(i+1, N, k+1)]
        p4 = pnums[get_point_index(i, N, k+1)]
        mesh.Add(Element2D(fd_boundary, [p1, p2, p3, p4]))
        face_count += 1

print(f"Added front and back faces: {face_count} faces so far")

Added front and back faces: 100 faces so far


In [10]:
# Left face (x=0)
for k in range(N):
    for j in range(N):
        p1 = pnums[get_point_index(0, j, k)]
        p2 = pnums[get_point_index(0, j+1, k)]
        p3 = pnums[get_point_index(0, j+1, k+1)]
        p4 = pnums[get_point_index(0, j, k+1)]
        mesh.Add(Element2D(fd_boundary, [p1, p2, p3, p4]))
        face_count += 1

# Right face (x=N)
for k in range(N):
    for j in range(N):
        p1 = pnums[get_point_index(N, j, k)]
        p2 = pnums[get_point_index(N, j+1, k)]
        p3 = pnums[get_point_index(N, j+1, k+1)]
        p4 = pnums[get_point_index(N, j, k+1)]
        mesh.Add(Element2D(fd_boundary, [p1, p4, p3, p2]))  # Reversed order for outward normal
        face_count += 1

print(f"Added left and right faces: {face_count} total boundary faces")

Added left and right faces: 150 total boundary faces


In [11]:
# Add boundary edges (12 edges of the cube)
edge_count = 0

# Bottom edges
for i in range(N):
    mesh.Add(Element1D([pnums[get_point_index(i, 0, 0)], pnums[get_point_index(i+1, 0, 0)]], index=1))
    mesh.Add(Element1D([pnums[get_point_index(i, N, 0)], pnums[get_point_index(i+1, N, 0)]], index=1))
    mesh.Add(Element1D([pnums[get_point_index(0, i, 0)], pnums[get_point_index(0, i+1, 0)]], index=1))
    mesh.Add(Element1D([pnums[get_point_index(N, i, 0)], pnums[get_point_index(N, i+1, 0)]], index=1))
    edge_count += 4

# Top edges
for i in range(N):
    mesh.Add(Element1D([pnums[get_point_index(i, 0, N)], pnums[get_point_index(i+1, 0, N)]], index=1))
    mesh.Add(Element1D([pnums[get_point_index(i, N, N)], pnums[get_point_index(i+1, N, N)]], index=1))
    mesh.Add(Element1D([pnums[get_point_index(0, i, N)], pnums[get_point_index(0, i+1, N)]], index=1))
    mesh.Add(Element1D([pnums[get_point_index(N, i, N)], pnums[get_point_index(N, i+1, N)]], index=1))
    edge_count += 4

# Vertical edges
for i in range(N):
    mesh.Add(Element1D([pnums[get_point_index(0, 0, i)], pnums[get_point_index(0, 0, i+1)]], index=1))
    mesh.Add(Element1D([pnums[get_point_index(N, 0, i)], pnums[get_point_index(N, 0, i+1)]], index=1))
    mesh.Add(Element1D([pnums[get_point_index(0, N, i)], pnums[get_point_index(0, N, i+1)]], index=1))
    mesh.Add(Element1D([pnums[get_point_index(N, N, i)], pnums[get_point_index(N, N, i+1)]], index=1))
    edge_count += 4

print(f"Added {edge_count} boundary edges")

Added 60 boundary edges


In [12]:
# Save the mesh
# mesh.Save("cube_mesh.vol")
# print("Mesh saved as 'cube_mesh.vol'")

# Print summary
print(f"\n3D cube mesh summary:")
print(f"- Grid size: {N}×{N}×{N}")
print(f"- Total points: {(N+1)**3}")
print(f"- Hexahedral elements: {N**3}")
print(f"- Boundary faces: {6 * N**2}")
print(f"- Boundary edges: {12 * N}")


3D cube mesh summary:
- Grid size: 5×5×5
- Total points: 216
- Hexahedral elements: 125
- Boundary faces: 150
- Boundary edges: 60


In [13]:
# Visualize the mesh
mesh = ngsolve.Mesh(mesh)
Draw(mesh, settings={"camera": {"pos": [2, 2, 2]}})

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {'camera': {'pos': [2, 2, 2]}…

BaseWebGuiScene